<a href="https://colab.research.google.com/github/HannaKi/Exploring_StatisticsFinland_zip_code_data/blob/main/zip_code_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [513]:
! pip install pyproj

In [514]:
import pandas as pd 
import numpy as np
import json
import pyproj
import plotly.express as px
import pprint

# Data

In [515]:
# https://stat.fi/static/media/uploads/org/avoindata/kartta-aineistojen_lataus_url-osoitteen_kautta_ohje.pdf
%%bash
wget -nc --quiet "https://geo.stat.fi/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=json"
mv "/content/wfs?service=WFS&version=2.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=json" "paavo_polygon.json"

wget -nc --quiet "https://geo.stat.fi/geoserver/wfs?service=WFS&version=2.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=csv"
mv "/content/wfs?service=WFS&version=2.0.0&request=GetFeature&typeName=postialue:pno_tilasto&outputFormat=csv" "paavo_data.csv"

## Inspect the data

### Data

In the data we have coordinates and demographics. [Data description](https://www.stat.fi/tup/paavo/tietosisalto_ja_esimerkit.html#tietosisalto) (only in Finnish).

In [516]:
df=pd.read_csv('paavo_data.csv', sep=',')
print(df.info())
with pd.option_context('display.max_columns', None):
  display(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3027 entries, 0 to 3026
Columns: 114 entries, FID to pt_muut
dtypes: float64(3), int64(107), object(4)
memory usage: 2.6+ MB
None


,FID,geom,postinumeroalue,nimi,namn,euref_x,euref_y,pinta_ala,vuosi,kunta,he_vakiy,he_naiset,he_miehet,he_kika,he_0_2,he_3_6,he_7_12,he_13_15,he_16_17,he_18_19,he_20_24,he_25_29,he_30_34,he_35_39,he_40_44,he_45_49,he_50_54,he_55_59,he_60_64,he_65_69,he_70_74,he_75_79,he_80_84,he_85_,ko_ika18y,ko_perus,ko_koul,ko_yliop,ko_ammat,ko_al_kork,ko_yl_kork,hr_tuy,hr_ktu,hr_mtu,hr_pi_tul,hr_ke_tul,hr_hy_tul,hr_ovy,te_taly,te_takk,te_as_valj,te_yks,te_nuor,te_eil_np,te_laps,te_plap,te_aklap,te_klap,te_teini,te_yhlap,te_aik,te_elak,te_omis_as,te_vuok_as,te_muu_as,tr_kuty,tr_ktu,tr_mtu,tr_pi_tul,tr_ke_tul,tr_hy_tul,tr_ovy,ra_ke,ra_raky,ra_muut,ra_asrak,ra_asunn,ra_as_kpa,ra_pt_as,ra_kt_as,ra_muu_as,tp_tyopy,tp_alku_a,tp_jalo_bf,tp_palv_gu,tp_a_maat,tp_b_kaiv,tp_c_teol,tp_d_ener,tp_e_vesi,tp_f_rake,tp_g_kaup,tp_h_kulj,tp_i_majo,tp_j_info,tp_k_raho,tp_l_kiin,tp_m_erik,tp_n_hall,tp_o_julk,tp_p_koul,tp_q_terv,tp_r_taid,tp_s_muup,tp_t_koti,tp_u_kans,tp_x_tunt,pt_vakiy,pt_tyoll,pt_tyott,pt_0_14,pt_opisk,pt_elakel,pt_muut
0,pno_tilasto.1,MULTIPOLYGON (((385653.89279999956 6671591.047...,100,Helsinki keskusta - Etu-Töölö,Helsingfors centrum - Främre Tölö,384979,6672361,2353278,2022,91,18373,9689,8684,41,390,466,709,348,224,294,1587,2382,1872,1433,1278,1060,1109,1081,880,778,944,808,391,339,16236,1776,14460,2621,2854,2962,6023,16236,45114,29258,2914,6572,6750,732469677,10380,1.7,39.0,5379,2226,822,1334,364,649,541,435,266,6674,2400,5026,5289,65,10380,70284,41921,1840,4098,4442,729552153,0,621,305,316,12341,65.8,3,11721,617,51616,27,2176,49413,27,0,885,719,0,572,6545,1032,4612,9109,4452,1200,8589,2054,2108,2291,2829,2180,2367,0,45,0,18334,10412,578,1850,1336,3375,783
1,pno_tilasto.2,MULTIPOLYGON (((385316.0920000002 6671076.9840...,120,Punavuori - Bulevardi,Rödbergen - Bulevarden,385531,6671434,414010,2022,91,7216,3792,3424,41,143,217,352,176,107,107,546,890,688,566,551,451,467,377,352,333,345,281,151,116,6221,679,5542,1039,1192,1036,2275,6221,48502,29684,1098,2444,2679,301732169,4047,1.8,39.4,2042,796,327,600,134,268,260,227,156,2544,919,2165,1840,42,4047,74856,43238,696,1566,1785,302943400,0,237,87,150,4777,69.5,7,4492,278,7203,3,189,7009,3,0,167,0,0,22,568,234,981,568,820,78,1228,277,419,351,496,167,804,1,17,2,7140,3990,236,831,468,1240,375
2,pno_tilasto.3,MULTIPOLYGON (((386212.11120000016 6671061.262...,130,Kaartinkaupunki,Gardesstaden,386244,6671474,428960,2022,91,1721,889,832,41,42,39,81,32,26,39,140,169,153,114,111,124,165,120,91,82,84,62,25,22,1501,159,1342,275,273,223,571,1501,64088,31558,316,477,708,96196739,948,1.8,44.4,445,161,80,150,38,66,65,49,30,597,205,480,460,8,948,101682,49730,183,284,481,96394652,0,140,103,37,1192,78.5,0,1000,192,11597,8,321,11268,8,4,140,2,4,171,460,115,880,1021,1334,243,2500,318,2033,172,647,273,1199,0,73,0,1582,887,30,175,121,268,101
3,pno_tilasto.7,MULTIPOLYGON (((386488.1311999997 6671737.1993...,170,Kruununhaka,Kronohagen,386384,6672361,713343,2022,91,7238,3880,3358,42,135,225,341,161,104,129,479,725,646,584,558,491,490,443,414,360,388,272,167,126,6272,647,5625,960,1122,1047,2496,6272,41559,30591,1060,2475,2737,260657698,4023,1.8,39.2,1964,680,268,620,126,285,269,213,138,2439,986,2199,1799,25,4023,64977,44706,642,1595,1786,261403168,0,256,121,135,4806,68.8,9,4589,208,9765,3,138,9624,3,0,59,0,1,78,216,34,467,247,644,51,445,89,3781,2047,406,666,514,0,17,0,7489,4224,227,871,533,1335,299
4,pno_tilasto.4,MULTIPOLYGON (((386577.04949999973 6670280.544...,140,Kaivopuisto - Ullanlinna,Brunnsparken - Ulrikasborg,386394,6670766,931841,2022,91,7995,4339,3656,42,189,263,387,178,106,120,573,783,705,633,593,494,481,512,446,393,439,358,190,152,6872,759,6113,1107,1259,1152,2595,6872,60034,31333,1197,2522,3153,412551855,4497,1.8,41.8,2263,773,346,697,176,334,301,218,154,2676,1141,2353,2102,42,4497,92033,45174,745,1622,2130,413873777,0,237,62,175,5391,74.1,10,5358,23,2532,5,136,2391,5,0,22,0,0,114,227,635,235,75,26,143,287,86,13,61,306,98,113,0,86,0,7958,4246,250,990,522,1528,422


In [517]:
# Filter the data for the demo and keep just Helsinki municipality
df=df[df.kunta==91]
df=df[['FID','postinumeroalue', 'euref_x', 'euref_y','nimi', 'ko_ika18y', 'ko_yl_kork']].copy()
# add new feature
df['korkeakoulutettujen_osuus']=100*(df.ko_yl_kork/df.ko_ika18y) # share of atleast 18 years old who have graduate degree
print(df.shape)
df.head()

(84, 8)


,FID,postinumeroalue,euref_x,euref_y,nimi,ko_ika18y,ko_yl_kork,korkeakoulutettujen_osuus
0,pno_tilasto.1,100,384979,6672361,Helsinki keskusta - Etu-Töölö,16236,6023,37.096576
1,pno_tilasto.2,120,385531,6671434,Punavuori - Bulevardi,6221,2275,36.569683
2,pno_tilasto.3,130,386244,6671474,Kaartinkaupunki,1501,571,38.041306
3,pno_tilasto.7,170,386384,6672361,Kruununhaka,6272,2496,39.795918
4,pno_tilasto.4,140,386394,6670766,Kaivopuisto - Ullanlinna,6872,2595,37.761932


In [518]:
df.isna().sum() # any missing values?

FID                          0
postinumeroalue              0
euref_x                      0
euref_y                      0
nimi                         0
ko_ika18y                    0
ko_yl_kork                   0
korkeakoulutettujen_osuus    1
dtype: int64

In [519]:
df[pd.isna(df.korkeakoulutettujen_osuus)] # This zip code area has no residents at all. 

,FID,postinumeroalue,euref_x,euref_y,nimi,ko_ika18y,ko_yl_kork,korkeakoulutettujen_osuus
5,pno_tilasto.13,230,384728,6677128,Helsinki postikeskus,0,0,NaN


Normaly I would drop the row above with NA but since I have corresponding area in the GeoJSON (see below) I will keep it and fill in the NA with zero. Plotly can not handle NA´s so they must be handled somehow.

In [520]:
df=df.fillna(0)

In [521]:
assert(len(df)==df.postinumeroalue.nunique())

### GeoJSON

For the specs of the data format see: https://geojson.org/

For the requirements of the Plotly choropleth map see: https://plotly.github.io/plotly.py-docs/generated/plotly.express.choropleth_mapbox.html

Some things that need to be taken care of: 

- The GeoJSON for the plotly choropleth map *must contain a Polygon feature collection*
- In a GeoJSON for type "Polygon", the "coordinates" member MUST be an array of linear ring coordinate arrays.
 - A linear ring is a closed LineString with four or more positions.
 - The first and last positions are equivalent, and they MUST contain identical values; their representation SHOULD also be identical.
 - A linear ring is the boundary of a surface or the boundary of a hole in a surface.
 - A linear ring MUST follow the right-hand rule with respect to the
      area it bounds, i.e., exterior rings are counterclockwise, and
      holes are clockwise.

In [522]:
with open('./paavo_polygon.json', 'r') as json_file:
    p = json.load(json_file)

In [523]:
print('The data is a GeoJSON of type:', p['type'])
print('Since it is a json, it has keys:')
print(p.keys())
print()
print('Features are in a list:',type(p['features'])) 
print('And each feature is a dictionary:',type(p['features'][0])) # list of dicts --> our features of interest are here

print('This key holds the value of the coordinate system used',p['crs']) # Coordinate reference system 


The data is a GeoJSON of type: FeatureCollection
Since it is a json, it has keys:
dict_keys(['type', 'features', 'totalFeatures', 'numberMatched', 'numberReturned', 'timeStamp', 'crs', 'bbox'])

Features are in a list: <class 'list'>
And each feature is a dictionary: <class 'dict'>
This key holds the value of the coordinate system used {'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:EPSG::3067'}}


In [524]:
# Nothing interesting here, delete keys for clarity...
p.pop('bbox')
p.pop('crs')
p.pop('totalFeatures')
p.pop('numberReturned')
p.pop('numberMatched')
p.pop('timeStamp')
p.keys()

dict_keys(['type', 'features'])

In [525]:
pp = pprint.PrettyPrinter()
# print the first dictionary in the list of features to see the keys and associated values
# pp.pprint(p['features'][0])

In [526]:
# Let's filter down the data at first. Let's keep Helsinki municipality only.
p['new_features']=[]
for feat in p['features']:
  if feat['properties']['kunta']=='091':
    p['new_features'].append(feat)

assert(len(p['new_features'])==len(df)) # Sanity check

p['features']=p['new_features']
p.pop('new_features')
p.keys()

dict_keys(['type', 'features'])

We need to convert the coordinates to WSG84 so we can plot them with Plotly and take care that the data is of type Polygon.

In [527]:
pp.pprint(p['features'][0]['geometry']['type']) # This needs to be changed!

'MultiPolygon'


Coordinates are in lists which are inside a nested list:

In [528]:
print(type(p['features'][0]['geometry']['coordinates']))
print(type(p['features'][0]['geometry']['coordinates'][0]))
print(type(p['features'][0]['geometry']['coordinates'][0][0]))
print('Here is one point:', p['features'][0]['geometry']['coordinates'][0][0][0], 'still in a',type(p['features'][0]['geometry']['coordinates'][0][0][0]))
print(type(p['features'][0]['geometry']['coordinates'][0][0][0][0]), p['features'][0]['geometry']['coordinates'][0][0][0][0])

<class 'list'>
<class 'list'>
<class 'list'>
Here is one point: [385653.8928, 6671591.0475] still in a <class 'list'>
<class 'float'> 385653.8928


For the transformation we need to know the EPSG-codes: 

https://epsg.io/3067: EPSG:3067 equals ETRS89 / TM35FIN(E,N) -- Finland. This is where we start from

https://epsg.io/4326: EPSG:4326 equals WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS. This is our goal.

Let's make a transformer with pyproj library:

In [529]:
# https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
test=p['features'][0]['geometry']['coordinates'][0][0][0]
print(test)
transformer = pyproj.Transformer.from_crs("epsg:3067", "epsg:4326")
transformer.transform(test[0],test[1]) # seems good!

[385653.8928, 6671591.0475]


(60.16518005365167, 24.939463163395455)

In [530]:
# 5 first coordinates of one polygon before transformation:
i=0
while i<5:
  print(p['features'][0]['geometry']['coordinates'][0][0][i])
  i+=1
print('And the type of the GeoJSON Object:', p['features'][0]['geometry']['type'])  

[385653.8928, 6671591.0475]
[385573.6195, 6671699.8005]
[385612.6593, 6671729.6573]
[385575.7769, 6671779.0565]
[385548.6877, 6671753.9019]
And the type of the GeoJSON Object: MultiPolygon


Let's loop all the features and do the transfomations and other data munging.

In [531]:
for feat in p['features']:
  # new feature id:
  #feat['properties'] = {'unique_id': feat['id']} # key of the feature. This is for Plotly
  # coordinate transformations
  coord_feat_reversed=[]
  feat['geometry']['type']='Polygon' # over ride type "Multipolygon" with "Polygon"
  for c in feat['geometry']['coordinates']:
    wgs84_list=[]
    for i in c:
      for j in i:
        wgs84=transformer.transform(j[0], j[1]) # convert from EPSG:3067 to EPSG:4326
        wgs84=list(wgs84)
        rounded=[round(x, 6) for x in wgs84]
        rounded=list((rounded[1], rounded[0])) # round unnecessary decimals (too accurate) AND switch places of lat and lon in the list for Plotly
        wgs84_list.append(rounded)
    rev=list(reversed(wgs84_list)) # revert the list so that requirement of counterclocwise sequence coordinate points in Polygon is met
  coord_feat_reversed.append(rev)
  feat['geometry']['coordinates']=coord_feat_reversed # store the new values

In [532]:
print('5 first coordinates of one polygon after transformation:')
i=0
while i<5:
  print(p['features'][0]['geometry']['coordinates'][0][i])
  i+=1
print('And the type of the GeoJSON Object:', p['features'][0]['geometry']['type'])  

5 first coordinates of one polygon after transformation:
[24.939463, 60.16518]
[24.941682, 60.16595]
[24.940285, 60.166956]
[24.939578, 60.166817]
[24.939064, 60.167122]
And the type of the GeoJSON Object: Polygon


In [533]:
#pp.pprint(p['features'][0])

# Map

When plotting with Plotly we need to pass keys to match the GeoJSON and the data frame. The key of the data frame is given to "locations" and the key of GeoJSON to "featureidkey". 

In this case both have the same value of zip code name.

In [546]:
fig = px.choropleth_mapbox(
    data_frame=df,
    geojson=p,                      # Must contain a Polygon feature collection, with IDs, which are references from locations
    locations=df.FID,               # A name of a column in data_frame. Values from this column or array_like are to be interpreted according to locationmode and mapped to longitude/latitude.
    featureidkey='id',              # Path to field in GeoJSON feature object with which to match the values passed in to locations
    mapbox_style="carto-positron", 
    color=df.korkeakoulutettujen_osuus,
    zoom=10,
    height=600,
    color_continuous_scale='magma', # https://plotly.com/python/builtin-colorscales/
    title='Share of graduate degrees',
    labels={'korkeakoulutettujen_osuus':'%'},
    hover_data=['nimi','korkeakoulutettujen_osuus'],
    opacity=0.7,
    center={ 
        'lat': 60.226995,  # Sets the center point of the map.
        'lon': 25.035727
    })
fig.update_geos(fitbounds='locations', visible=False)
fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
#fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0}) # to remove the space for title
# fig.show(renderer='notebook') # for VS Code
fig.show()

Sources for code: 

https://towardsdatascience.com/interactive-map-using-geographical-information-in-plotly-express-362081457600

https://medium.com/tech-carnot/plotly-mapbox-interactive-choropleth-visualization-tutorial-957dcdbca90b

https://stackoverflow.com/a/67637994